In [2]:
import numpy as np
from collections import Counter

def loadDataSet(filename):
    dataMat=[]
    fr = open(fileName)
    for line in fr.readlines():
        curline = line.strip().split('\t')
        fltline = map(float,curLine)
        dataMat.append(fltline)
    return dataMat


In [3]:
def binSplitDataSet(dataSet,feature,value):
    # print(dataSet[:,feature]>value)
    nonzeroData_1 = np.nonzero(dataSet[:,feature]>value)
    nonzeroData_2 = np.nonzero(dataSet[:,feature]<=value)
    # mat0 = dataSet[nonzeroData_1[0],:][0]  原书有误，去掉最后的[0] 
    # mat1 = dataSet[nonzeroData_2[0],:][0]  同理原书有误，去掉最后的[0] 
    mat0 = dataSet[nonzeroData_1[0],:]
    mat1 = dataSet[nonzeroData_2[0],:]
    # print("nonzeroData_1",nonzeroData_1)
    # print("nonzeroData_2:",nonzeroData_2)
    return mat0,mat1

testMat=np.mat(np.eye(4))
mat0,mat1=binSplitDataSet(testMat,1,0.5)
print("mat0:",mat0)
print("mat1:",mat1)

mat0: [[0. 1. 0. 0.]]
mat1: [[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [62]:
def regLeaf(dataSet):
    return np.mean(dataSet[:,-1])

def classifyLeaf(dataSet):
    # return np.argmax(np.bincount(dataSet[:,-1].flatten()))
    counts = Counter(dataSet[:,-1].flatten())
    return counts.most_common(1)[0][0]
    
def regErr(dataSet):
    return np.var(dataSet[:,-1])*np.shape(dataSet)[0] 

import math
def giniErr(dataSet):
    counts = Counter(dataSet[:,-1].flatten())
    jini = 1
    count_all = len(dataSet)
    for label,count in counts.items():
        # print(label,count)
        p = float(count)/float(count_all)
        # print("p:",p)
        jini = jini-math.pow(float(count)/float(count_all),2)
        # print("jini",jini)
    return jini*count_all
    
counts = Counter([-1,-1,-1,1,2,7,2,3,1])
print(counts)
print(counts.most_common(1))
# test = np.bincount([-1,-1,-1,1,2,7,2,3,1])
# print(test)
# print(np.argmax(test))
test_dataSet = np.array([[1,1],[2,1],[1,0]])
giniErr(test_dataSet)

Counter({-1: 3, 1: 2, 2: 2, 7: 1, 3: 1})
[(-1, 3)]


1.3333333333333335

In [10]:
def chooseBestSplit(dataSet,leafType=regLeaf,errType=regErr,ops=(1,2)): 
    dataSet=np.array(dataSet)  #dataSet 统一转换为array
    tolS = ops[0];tolN = ops[1]
    # print((dataSet[:,-1].flatten().tolist()))
    if len(set(dataSet[:,-1].flatten().tolist()))==1:
    #if len(set(dataSet[:,-1].T.tolist()[0]))==1: #判断1，unhashable type: 'matrix'，所以要转置，再转为list
        return None,leafType(dataSet)
    m,n = np.shape(dataSet)
    S = errType(dataSet)
    bestS = np.inf;bestIndex = 0;bestValue = 0
    for featIndex in range(n-1):
        # for splitVal in set(dataSet[:,featIndex]): 原书中有误，没有T.tolist
        for splitVal in set(dataSet[:,featIndex].T.tolist()):
            # print("featIndex",featIndex,"splitVal",splitVal)
            
            mat0, mat1 = binSplitDataSet(dataSet,featIndex,splitVal)
            if (np.shape(mat0)[0]<tolN) or (np.shape(mat1)[0]<tolN): #判断2，根据splitVal切分，如果切分为二后，有一类的的记录少于tolN，则跳过
                continue 
                
            newS =errType(mat0) + errType(mat1)  #计算二分之后的误差和
            # print("bestS:",newS)
            if newS < bestS:   
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
                
    if (S-bestS)< tolS:  #如果误差减少不大则退出
        # print("small than tolS ")
        return None, leafType(dataSet)
    
    mat0,mat1= binSplitDataSet(dataSet,bestIndex,bestValue)
    if np.shape(mat0)[0]<tolN or np.shape(mat1)[0]<tolN: 
        # 如果最优bestIndex,bestValue切分后，有一类的的记录少于tolN，则退出,这种情况发生只有一种可能，所有情况组合切分后，判断2都为真，
        # 一直continue，所以bestIndex，bestValue都等于初始值0，代入binSplitDataSet后，该判断也为真
        return None, leafType(dataSet)
#   原书中的以上三条代码可以由以下语句替换，逻辑更为清楚
#     if beatS == np.inf:
#         return None, leafType(dataSet)
    return bestIndex,bestValue

dataSet=np.mat([[1,2,3,4],[2,3,1,4],[0,1,3,3]])
print(dataSet)
print(chooseBestSplit(dataSet,ops=(100,2)))


[[1 2 3 4]
 [2 3 1 4]
 [0 1 3 3]]
(None, 3.6666666666666665)


In [68]:
def createTree(dataSet,leafType=regLeaf,errType=regErr,ops=(1,2),layer=999): #对 ,
    feat,val = chooseBestSplit(dataSet,leafType,errType,ops)
    if feat == None: return val
    if layer ==0:return leafType(dataSet)
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet,rSet = binSplitDataSet(dataSet,feat,val)
    retTree['left']=createTree(lSet,leafType,errType,ops,layer=layer-1)
    retTree['right']=createTree(rSet,leafType,errType,ops,layer=layer-1)
    return retTree

In [69]:
def regTreeEval(model, inDat):
    return float(model)

def isTree(obj):
    return (type(obj).__name__=='dict')

In [70]:
def treeForeCast(tree, inData, modelEval=regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)

In [76]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
iris_dataset=datasets.load_iris()
data_x = iris_dataset.data
data_y = iris_dataset.target
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,random_state=0,test_size=0.25)
# print(x_train)
# print(y_train)

In [80]:
dataSet = np.hstack((x_train,y_train.reshape((len(y_train),1))))
print(dataSet)
retTree = createTree(dataSet,leafType=classifyLeaf,errType=giniErr,ops=(0,1),layer=2) #leafType=classifyLeaf，归类
print(retTree)
y_pred_train=[treeForeCast(retTree,i) for i in x_train]
# print(y_pred_train)
print('Test set score:{:.5f}'.format(np.mean(y_pred_train==y_train)))

y_pred=[treeForeCast(retTree,i) for i in x_test]
print(y_pred)
print('Test set score:{:.5f}'.format(np.mean(y_pred==y_test))) 

[[5.9 3.  4.2 1.5 1. ]
 [5.8 2.6 4.  1.2 1. ]
 [6.8 3.  5.5 2.1 2. ]
 [4.7 3.2 1.3 0.2 0. ]
 [6.9 3.1 5.1 2.3 2. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.  2.  3.5 1.  1. ]
 [6.5 3.  5.5 1.8 2. ]
 [6.7 3.3 5.7 2.5 2. ]
 [6.  2.2 5.  1.5 2. ]
 [6.7 2.5 5.8 1.8 2. ]
 [5.6 2.5 3.9 1.1 1. ]
 [7.7 3.  6.1 2.3 2. ]
 [6.3 3.3 4.7 1.6 1. ]
 [5.5 2.4 3.8 1.1 1. ]
 [6.3 2.7 4.9 1.8 2. ]
 [6.3 2.8 5.1 1.5 2. ]
 [4.9 2.5 4.5 1.7 2. ]
 [6.3 2.5 5.  1.9 2. ]
 [7.  3.2 4.7 1.4 1. ]
 [6.5 3.  5.2 2.  2. ]
 [6.  3.4 4.5 1.6 1. ]
 [4.8 3.1 1.6 0.2 0. ]
 [5.8 2.7 5.1 1.9 2. ]
 [5.6 2.7 4.2 1.3 1. ]
 [5.6 2.9 3.6 1.3 1. ]
 [5.5 2.5 4.  1.3 1. ]
 [6.1 3.  4.6 1.4 1. ]
 [7.2 3.2 6.  1.8 2. ]
 [5.3 3.7 1.5 0.2 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [6.4 2.7 5.3 1.9 2. ]
 [5.7 3.  4.2 1.2 1. ]
 [5.4 3.4 1.7 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [6.9 3.1 4.9 1.5 1. ]
 [4.6 3.1 1.5 0.2 0. ]
 [5.9 3.  5.1 1.8 2. ]
 [5.1 2.5 3.  1.1 1. ]
 [4.6 3.4 1.4 0.3 0. ]
 [6.2 2.2 4.5 1.5 1. ]
 [7.2 3.6 6.1 2.5 2. ]
 [5.7 2.9 4

In [81]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2) # random_state=0
tree.fit(x_train,y_train)
y_pred_train = tree.predict(x_train)
print('Test set score:{:.5f}'.format(np.mean(y_pred_train==y_train)))
y_pred = tree.predict(x_test)
print(y_pred)
print('Test set score:{:.5f}'.format(np.mean(y_pred==y_test)))

Test set score:0.96429
[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0 1 1 0 2 1 0 1 2 1 0
 2]
Test set score:0.89474
